In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

In [36]:
a = 349
a += 10 - 360*(a+10>360)
a
60+180 - 90

150

In [156]:
df = pd.read_csv ('data/newLogs/LOG3.csv', names=["time", "type","tag","text"])
mdf = pd.read_csv ('data/mpuLogs/LOG5.csv', names=["time", "type","tag","text"])

### Design the State Variables

For our robot we will maintain the position and orientation:

$$\mathbf x = \begin{bmatrix}x & \dot x &  \ddot{x} & y & \dot y &  \ddot{y} \end{bmatrix}^\mathsf{T}$$

I could include velocities into this model, but as you will see the math will already be quite challenging.

The control input $\mathbf{u}$ is the commanded velocity and steering angle

$$\mathbf{u} = \begin{bmatrix}v & \alpha\end{bmatrix}^\mathsf{T}$$

In [157]:
mpudf = pd.DataFrame(data=[np.asarray(el.split(), dtype=np.float64) for el in mdf.query("tag == 'mpu9250' and type == 4").text],
                    index = mdf.query("tag == 'mpu9250' and type == 4").time/1000,
                    columns=["heading", "pitch", "roll"])
#mpudf['heading'] += 90 - 360*(mpudf['heading']+90>360)
px.line(mpudf)

In [138]:
neo6mdf = pd.DataFrame(data=[(el["text"]+" "+ str(el['time']/1000)).split() for i,el in df.query("tag == 'neo6m' and type == 5").iterrows()],
                    columns=["lat", "lon", "cog","speed","variation","valid","time"],dtype=float)
neo6mdf['lat'] = neo6mdf['lat'].astype(float)
neo6mdf['lon'] = neo6mdf['lon'].astype(float)
fig = px.line_mapbox(neo6mdf, lat="lat", lon="lon",hover_data={"time","lat","lon","speed","cog"},text="time", zoom=3, height=600)     
fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=17, mapbox_center_lat = neo6mdf.lat[0],mapbox_center_lon=neo6mdf.lon[0],
    margin={"r":0,"t":0,"l":0,"b":0})
fig.show() 

In [144]:
earthRate =  0.000072921
earthRadius =  6378145.0
earthRadiusInv =   1.5678540e-7
latRef = np.deg2rad(neo6mdf.lat[0])
lonRef = np.deg2rad(neo6mdf.lon[0])
altRef = 500

def lla2ned(lat, lon, alt=0):
    lat, lon = np.deg2rad(lat), np.deg2rad(lon) 
    xN = earthRadius * (lat - latRef)
    yE = earthRadius * np.cos(latRef) * (lon - lonRef)
    zD = -(alt - altRef)
    return xN,yE,zD
def ned2lla(xN,yE,zD=0):
    lat = latRef + xN * earthRadiusInv
    lon = lonRef + yE * earthRadiusInv / np.cos(latRef)
    alt = altRef - zD
    return np.rad2deg(lat),np.rad2deg(lon),alt
def vel2ned(speed,cog,velD=0):
    cog = np.deg2rad(cog)
    vx = speed*np.cos(cog)
    vy = speed*np.sin(cog)
    vz = velD
    return vx,vy,vz
def ned2vel(vx,vy):
    return (vx**2+vy**2)**0.5

def fx(x, dt):
    F = np.array([[1, dt, 0, 0 ],
                  [0, 1,  0, 0 ], 
                  [0, 0,  1, dt],
                  [0, 0,  0, 1]])
    return F @ x

def hx(x):
    return [x[0],x[2],x[1],x[3]]

In [145]:
from filterpy.kalman import UnscentedKalmanFilter as UKF
from filterpy.common import Q_discrete_white_noise
from filterpy.kalman import MerweScaledSigmaPoints
from filterpy.stats import mahalanobis

dt = 0.4

sigmas = MerweScaledSigmaPoints(4, alpha=.1, beta=2., kappa=1.)
ukf = UKF(dim_x=4, dim_z=4, fx=fx,
          hx=hx, dt=dt, points=sigmas,)
ukf.x = np.array([0., 0., 0., 0.])
ukf.R = np.diag([6**2, 6**2,6**2,6**2])
ukf.P = np.diag([50,20,50,20]) 
ukf.Q = Q_discrete_white_noise(dim=2, dt=dt, var=.2,block_size=2)

In [146]:
last_t = 0
latarr,lonarr,velarr = [],[],[]
for index, row in neo6mdf.iterrows():
    dt = row['time'] - last_t
    last_t = row['time']
    ukf.Q = Q_discrete_white_noise(dim=2, dt=dt, var=.2,block_size=2)
    ukf.predict(dt)
    #mesto cog ke se koristi heading 
    # [row['heading'] for index, row in mpudf.iterrows() if (index>last_t)][0]
    vx,vy,_=vel2ned(row['speed'],134 if row['cog'] == 0 else row['cog'])
    x,y,_ = lla2ned(row['lat'],row['lon'])
    GATE_LIMIT = 4.
    std = np.sqrt(ukf.P.diagonal())
    err = [x-ukf.x[0],y-ukf.x[2]]
    if err[0] > GATE_LIMIT * std[0] or err[1] > GATE_LIMIT * std[2]:
        print(f'discarding measurement, error is {err[0]/std[0]:.0f} std, {err[1]/std[2]:.0f} std time: {last_t}')
        lat,lon,_ = ned2lla(ukf.x[0],ukf.x[2])
        latarr.append(lat)
        lonarr.append(lon)
        velarr.append(ned2vel(ukf.x[1],ukf.x[3]))
        continue
    ukf.update([x,y,vx,vy])
    lat,lon,_ = ned2lla(ukf.x[0],ukf.x[2])
    latarr.append(lat)
    lonarr.append(lon)
    velarr.append(ned2vel(ukf.x[1],ukf.x[3]))

discarding measurement, error is 6 std, -3 std time: 19.044
discarding measurement, error is 8 std, -3 std time: 19.282
discarding measurement, error is 9 std, -3 std time: 19.576
discarding measurement, error is 10 std, -3 std time: 19.81
discarding measurement, error is 11 std, -3 std time: 20.054
discarding measurement, error is 11 std, -3 std time: 20.292
discarding measurement, error is 13 std, -3 std time: 20.526
discarding measurement, error is 14 std, -2 std time: 20.688
discarding measurement, error is 15 std, -3 std time: 21.036
discarding measurement, error is 16 std, -3 std time: 21.266
discarding measurement, error is 16 std, -3 std time: 21.384
discarding measurement, error is 17 std, -3 std time: 21.674
discarding measurement, error is 18 std, -3 std time: 22.044
discarding measurement, error is 19 std, -3 std time: 22.156
discarding measurement, error is 19 std, -4 std time: 22.516
discarding measurement, error is 19 std, -4 std time: 22.812
discarding measurement, erro

In [147]:
ukfdf = neo6mdf.copy()
ukfdf['lat'] = latarr
ukfdf['lon'] = lonarr
ukfdf['speed'] = velarr
fig = px.line_mapbox(ukfdf, lat="lat", lon="lon",hover_data={"time","lat","lon","speed","cog"},text="time", zoom=3, height=600)     
fig.update_layout(mapbox_style="stamen-terrain", mapbox_zoom=17, mapbox_center_lat = ukfdf.lat[0],mapbox_center_lon=ukfdf.lon[0],
    margin={"r":0,"t":0,"l":0,"b":0})
fig.show()

In [148]:
px.line(y = [neo6mdf.speed,ukfdf.speed],x=ukfdf.time)

In [21]:

px.line(x=neo6mdf.time,y=neo6mdf.cog)

In [21]:
latRef

0.7282518948969492

In [87]:
vx,vy,vz = vel2ned(neo6mdf.speed[0],151,1)

In [88]:
x,y,z= lla2ned(neo6mdf.lat,neo6mdf.lon,20)

In [89]:
xn = x+100 + 0.441 * vx
yn = y+100 + 0.441 * vy

In [90]:
latn,lotn,altn = ned2lla(xn,yn,z)

In [86]:
neo6mdf.lat = latn
neo6mdf.lon = lotn